In [165]:
import os
import pandas as pd

In [256]:
def clean_and_correct_header(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpa o DataFrame removendo linhas não necessárias e define o cabeçalho correto.

    Args:
        df (pd.DataFrame): O DataFrame original extraído de um arquivo.

    Returns:
        pd.DataFrame: O DataFrame limpo e com o cabeçalho corrigido.
    """
    # Remover linhas vazias ou indesejadas
    df = df.dropna(how="all").reset_index(drop=True)  # Remove linhas onde todos os valores são NaN

    # Tentar encontrar a linha que representa o cabeçalho real
    first_valid_index = df.index[df.notnull().all(axis=1)][0]  # Encontra a primeira linha não-nula
    df.columns = df.iloc[first_valid_index]  # Usa essa linha como cabeçalho
    df = df.drop(range(first_valid_index + 1))  # Remove as linhas acima do cabeçalho
    # Substituir o cabeçalho por um padrão correto se o extraído for inválido
    if 'Valor_(R$)' not in df.columns:
        expected_header = [
            "itens_fatura", "unid", "quant", "preco_unitario_br", "valor",
            "pis/cofins", "base_calculo_icms_br", "aliquota_icms_pp", "icms", "tarifa_unitaria_br"
        ]
        df.columns = expected_header

    # Remover colunas ou linhas indesejadas
    df = df[~df.apply(lambda row: row.astype(str).str.contains(';;;|com tributos|ICMS', case=False).any(), axis=1)]

    return df.reset_index(drop=True)

def filter_until_subtotal(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra o DataFrame até a linha que contém 'Subtotal Faturamento'.
    """
    subtotal_index = df[df.apply(lambda row: row.astype(str).str.contains('Subtotal Faturamento', case=False).any(), axis=1)].index
    return df.loc[:subtotal_index[0] - 1] if not subtotal_index.empty else df

def ajustar_sinal(valor):
    # Verifica se o valor é uma string e se termina com '-'
    if isinstance(valor, str) and valor.endswith('-'):
        # Remove o sinal '-' do final e o coloca no início
        return f"-{valor[:-1]}"
    return valor

def convert_to_numeric(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Converte os dados de colunas especificadas para tipos numéricos, ajustando
    valores com vírgula como separador decimal e sinal negativo no final.

    Args:
        df (pd.DataFrame): O DataFrame original.
        columns (list): Lista de colunas a serem convertidas.

    Returns:
        pd.DataFrame: O DataFrame com as colunas convertidas para numérico.
    """

    def convert_value(value):
        """
        Converte um valor no formato string (ex: '105,47-', '6,46', '-0,35874') para float.
        """
        value = value.strip()  # Remove espaços em branco ao redor

        # Verifica se o valor possui o sinal negativo no final e move para a frente
        if value.endswith('-'):
            value = '-' + value[:-1]

        # Substitui a vírgula pelo ponto para converter em float
        value = value.replace(',', '.')

        # Tenta converter para float, se falhar retorna NaN
        try:
            return float(value)
        except ValueError:
            return pd.NA

    # Aplica a conversão a cada coluna especificada
    for column in columns:
        df[column] = df[column].astype(str).apply(convert_value)

    return df

def calcular_percentual_cip(df: pd.DataFrame) -> pd.DataFrame:
    """
    Insere uma nova coluna chamada 'percentual_cip_pp' e calcula o percentual da linha 'CIP ILUM PUB PREF MUNICIPAL'
    com base na soma das linhas 'Energia Ativa Fornecida TE' e 'Energia Ativa Fornecida TUSD' da coluna 'valor'.

    Args:
        df (pd.DataFrame): O DataFrame original.

    Returns:
        pd.DataFrame: O DataFrame atualizado com a coluna 'percentual_cip_pp'.
    """

    # Função para converter o valor de string com vírgula para float para cálculos
    def to_float(value):
        try:
            return float(value.replace(',', '.'))
        except ValueError:
            return 0

    # Somar os valores de 'Energia Ativa Fornecida TE' e 'Energia Ativa Fornecida TUSD' sem alterar o DataFrame original
    valor_te = df.loc[df['itens_fatura'].str.contains('Energia Ativa Fornecida TE', case=False), 'valor'].apply(to_float).sum()
    valor_tusd = df.loc[df['itens_fatura'].str.contains('Energia Ativa Fornecida TUSD', case=False), 'valor'].apply(to_float).sum()

    # Calcular a soma total de TE e TUSD
    valor_total_te_tusd = valor_te + valor_tusd

    # Pegar o valor da linha 'CIP ILUM PUB PREF MUNICIPAL'
    valor_cip = df.loc[df['itens_fatura'].str.contains('CIP ILUM PUB PREF MUNICIPAL', case=False), 'valor'].apply(to_float).sum()

    # Calcular o percentual de CIP em relação ao total de TE e TUSD
    if valor_total_te_tusd > 0:
        percentual_cip_pp = (valor_cip / valor_total_te_tusd) * 100
    else:
        percentual_cip_pp = 0

    # Criar a coluna 'percentual_cip_pp' e inserir o valor calculado apenas na linha 'CIP ILUM PUB PREF MUNICIPAL'
    df['percentual_cip_pp'] = None
    df.loc[df['itens_fatura'].str.contains('CIP ILUM PUB PREF MUNICIPAL', case=False), 'percentual_cip_pp'] = f"{percentual_cip_pp:.2f}".replace('.', ',')

    return df

def ajustar_dados_csv(df: pd.DataFrame) -> pd.DataFrame:
    # Garantir que valores numéricos estão no formato correto (vírgula como separador decimal)
    for col in ['preco_unitario_br', 'valor', 'pis/cofins', 'base_calculo_icms_br', 'icms', 'tarifa_unitaria_br', 'percentual_cip_pp']:
        df[col] = df[col].astype(str).str.replace('.', ',')

    # Preencher valores ausentes em múltiplas colunas
    colunas_para_preencher = ['preco_unitario_br', 'percentual_cip_pp']
    df[colunas_para_preencher] = df[colunas_para_preencher].fillna("0")

    # Verificar e reorganizar as colunas
    df = df[['itens_fatura', 'unid', 'quant', 'preco_unitario_br', 'valor',
             'pis/cofins', 'base_calculo_icms_br', 'aliquota_icms_pp', 'icms', 'tarifa_unitaria_br', 'percentual_cip_pp']]

    return df

def ajustar_e_converter_dados_csv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ajusta os dados do DataFrame, preenche valores ausentes em colunas especificadas,
    converte colunas numéricas de string para float e organiza o DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame original.

    Returns:
        pd.DataFrame: O DataFrame ajustado e com as colunas numéricas convertidas.
    """

    # Converter colunas numéricas para float (substituindo vírgula por ponto)
    colunas_numericas = ['preco_unitario_br', 'valor', 'pis/cofins', 'base_calculo_icms_br', 'icms', 'tarifa_unitaria_br', 'percentual_cip_pp']

    for col in colunas_numericas:
        df[col] = df[col].str.replace(',', '.').astype(float)

    # Remover o símbolo de porcentagem (%) e converter para float
    df['aliquota_icms_pp'] = df['aliquota_icms_pp'].str.replace('%', '').str.replace(',', '.').astype(float)

    # Preencher valores ausentes em colunas específicas
    colunas_para_preencher = ['preco_unitario_br', 'percentual_cip_pp']
    df[colunas_para_preencher] = df[colunas_para_preencher].fillna(0)

    # Verificar e reorganizar as colunas
    df = df[['itens_fatura', 'unid', 'quant', 'preco_unitario_br', 'valor',
             'pis/cofins', 'base_calculo_icms_br', 'aliquota_icms_pp', 'icms', 'tarifa_unitaria_br', 'percentual_cip_pp']]

    return df

def salvar_df_como_csv(df: pd.DataFrame, nome_arquivo: str, caminho: str = './') -> None:
    """
    Salva o DataFrame no formato CSV.

    Args:
        df (pd.DataFrame): O DataFrame a ser salvo.
        nome_arquivo (str): O nome do arquivo CSV a ser salvo.
        caminho (str): O caminho onde o arquivo CSV será salvo. Por padrão, salva no diretório atual.
    """
    # Verificar se o nome do arquivo tem a extensão .csv
    if not nome_arquivo.endswith('.csv'):
        nome_arquivo += '.csv'

    # Definir o caminho completo para salvar o arquivo
    caminho_completo = f"{caminho}/{nome_arquivo}"

    # Salvar o DataFrame como CSV
    df.to_csv(caminho_completo, sep=';', index=False)
    print(f"Arquivo CSV salvo em: {caminho_completo}")

In [257]:
df = pd.read_csv("09_24.csv", delimiter=";")

# Lista das colunas a serem ajustadas
colunas_para_ajustar = ['preco_unitario_br', 'valor', 'pis/cofins', 'base_calculo_icms_br', 'icms']

df = clean_and_correct_header(df)
df = filter_until_subtotal(df)
df = filter_until_subtotal(df)
df[colunas_para_ajustar] = df[colunas_para_ajustar].map(ajustar_sinal)
# df_cleaned = convert_to_numeric(df, colunas_para_ajustar)
df = calcular_percentual_cip(df)

# df = ajustar_dados_csv(df)
df = ajustar_e_converter_dados_csv(df)


In [258]:
df
# print(df.dtypes)
# df.info()

# salvar_df_como_csv(df, "resultado_cip", "./files")

,itens_fatura,unid,quant,preco_unitario_br,valor,pis/cofins,base_calculo_icms_br,aliquota_icms_pp,icms,tarifa_unitaria_br,percentual_cip_pp
0,Energia Ativa Fornecida TE,kWh,324,0.35870,116.22,4.57,116.22,20.0,23.24,0.27291,0.00
1,Energia Ativa Fornecida TUSD,kWh,324,0.59065,191.37,7.54,191.37,20.0,38.27,0.44929,0.00
2,Energia Atv Inj TE mUC 09/2024 mPT,kWh,294,-0.35874,-105.47,-4.15,-105.47,20.0,-21.09,0.27291,0.00
3,Energia Atv Inj TUSD mUC 09/2024 mPT,kWh,294,-0.47255,-138.93,-6.84,0.00,0.0,0.00,0.44929,0.00
4,Adic Band Vermelha Comp,kWh,294,-0.01997,-5.87,-0.23,-5.87,20.0,-1.17,0.01525,0.00
5,Adicional Band Vermelha,kWh,324,0.01994,6.46,0.24,6.46,20.0,1.29,0.01522,0.00
6,CIP ILUM PUB PREF MUNICIPAL,NaN,NaN,0.00000,68.87,0.00,0.00,0.0,0.00,NaN,22.39
7,Juros Moratórios,NaN,NaN,0.00000,0.54,0.00,0.00,0.0,0.00,NaN,0.00
8,Multa,NaN,NaN,0.00000,1.26,0.00,0.00,0.0,0.00,NaN,0.00
